In [1]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam


In [2]:
base_dir = 'dataset'  # dataset folder containing all fruit categories
data_dir = 'data'      # Directory to save split datasets

# Get the list of classes (subfolder names)
classes = os.listdir(base_dir)

# Create directories for train, validation, and test
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')

# Create the directories
for folder in [train_dir, validation_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)
    for cls in classes:
        os.makedirs(os.path.join(folder, cls), exist_ok=True)

# Split the images into train, validation, and test sets
for cls in classes:
    cls_folder = os.path.join(base_dir, cls)
    images = os.listdir(cls_folder)

    # Splitting the dataset into train, validation, and test
    train_val, test = train_test_split(images, test_size=0.2, random_state=42)
    train, validation = train_test_split(train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2 of total data

    # Move images to the corresponding directories
    for img in train:
        shutil.copy(os.path.join(cls_folder, img), os.path.join(train_dir, cls, img))
    for img in validation:
        shutil.copy(os.path.join(cls_folder, img), os.path.join(validation_dir, cls, img))
    for img in test:
        shutil.copy(os.path.join(cls_folder, img), os.path.join(test_dir, cls, img))

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for validation
validation_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values for validation

# Data generator for testing
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values for testing

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize to match VGG16 input
    batch_size=16,
    class_mode='categorical'  
)

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),  # Resize to match VGG16 input
    batch_size=32,
    class_mode='categorical' 
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Resize to match VGG16 input
    batch_size=32,
    class_mode='categorical' 

Found 11324 images belonging to 30 classes.
Found 3851 images belonging to 30 classes.
Found 3846 images belonging to 30 classes.


In [ ]:
def create_model(num_classes):
    input_tensor = Input(shape=(224, 224, 3))

    # Use a pre-trained model (VGG16)
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model.trainable = False  # Freeze the base model

    # Add custom layers
    x = Flatten()(base_model.output)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)  # Single output for combined classification

    # Combine into a model
    model = Model(inputs=input_tensor, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create the model
num_classes = len(classes)
model = create_model(num_classes)

# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50 # Adjust the number of epochs as needed
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model
model.save('FinalFreshness.h5')

C:\Users\Prinshu Kumar\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
523/707 ━━━━━━━━━━━━━━━━━━━━ 7:03 2s/step - accuracy: 0.3714 - loss: 2.4283

C:\Users\Prinshu Kumar\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


707/707 ━━━━━━━━━━━━━━━━━━━━ 2041s 3s/step - accuracy: 0.4030 - loss: 2.2793 - val_accuracy: 0.6542 - val_loss: 1.2018
Epoch 2/50
  1/707 ━━━━━━━━━━━━━━━━━━━━ 21:13 2s/step - accuracy: 0.5000 - loss: 1.8054

C:\Users\Prinshu Kumar\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


707/707 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5000 - loss: 1.8054 - val_accuracy: 0.6364 - val_loss: 1.2029
Epoch 3/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 1820s 3s/step - accuracy: 0.6147 - loss: 1.4899 - val_accuracy: 0.6831 - val_loss: 1.3119
Epoch 4/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6250 - loss: 1.9845 - val_accuracy: 0.8182 - val_loss: 1.2060
Epoch 5/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 1789s 3s/step - accuracy: 0.6448 - loss: 1.5311 - val_accuracy: 0.7417 - val_loss: 1.0804
Epoch 6/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6250 - loss: 2.3742 - val_accuracy: 0.8182 - val_loss: 0.5007
Epoch 7/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 1878s 3s/step - accuracy: 0.6551 - loss: 1.6308 - val_accuracy: 0.7518 - val_loss: 1.0397
Epoch 8/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8750 - loss: 1.6323 - val_accuracy: 0.5455 - val_loss: 1.6547
Epoch 9/50
238/707 ━━━━━━━━━━━━━━━━━━━━ 21:09 3s/step - accuracy: 0.6827 - loss: 1.5526

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

# Visualize the model architecture and save the plot
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture saved as model_architecture.png")

# Plot training & validation accuracy and loss
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

# Plot the training history
plot_training_history(history)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
